## Train and evaluate model using Scikit-learn and Keras wrapper 

In [1]:
import tensorflow as tf
from aMNWtModel import AMNWtModel
from aMRNNModel import AMRNNModel


model_class = AMRNNModel
# model_class = AMNWtModel

seed = 10
# np.random.seed(seed=seed)  
tf.random.set_seed(seed)

# dataset_file = "./Dataset/IJN1463_EXP_UB_Anne.npz"
# objective=['BIOMASS_KT2440_WT3']

# dataset_file = "./Dataset/IJN1463_10_UB.npz"
# objective=['BIOMASS_KT2440_WT3']

dataset_file = "./Dataset/e_coli_core_UB_100.npz"
objective=['BIOMASS_Ecoli_core_w_GAM']
epoch = 200
batch_size = 7


# dataset_file = "./Dataset/biolog_iML1515_EXP_UB.npz"
# objective=['BIOMASS_Ec_iML1515_core_75p37M']
# epoch = 20
# batch_size = 30



print("---------------------------------------- model ----------------------------------------")
model = model_class(dataset_file=dataset_file, 
                   objective=objective,
                   timestep=4,
                #    n_hidden=1, 
                   hidden_dim=50,
                   epochs=50, 
                   verbose=True,
                   batch_size=7)
model.printout()

# Preprocessing
from sklearn.preprocessing import MinMaxScaler,StandardScaler,MaxAbsScaler 
from tools import MaxScaler
scaler= MaxScaler()
model.train_test_split(test_size=0.1, random_state=seed)
model.preprocess(scaler)
model.preprocessing_for_specific_model()

---------------------------------------- model ----------------------------------------
number of metabolites:  72
filtered measurements size:  1
dataset file: ./Dataset/e_coli_core_UB_100.npz
model type: AMNWt
model medium bound: UB
timestep: 4
training set size (100, 20) (100, 1)
training epochs: 50
training regression: True
training batch size: 7
training validation iter: 0
training early stopping: False


In [2]:
# cross validation
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold, cross_validate
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

estimator= KerasRegressor(build_fn=model.build_model, 
                          epochs=epoch, 
                          batch_size=batch_size, 
                          verbose=0)

scoring = {"loss_constraint":make_scorer(model.loss_constraint),
           "mse":make_scorer(model.mse),
           "R2":make_scorer(model.R2),
           }



callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
fit_params = {'callbacks': [callback]}
fit_params = {}

kfold= KFold(n_splits=5,shuffle=True, random_state=seed)

results=cross_validate(estimator, 
                       model.X_train, 
                       model.Y_train, 
                       cv=kfold, 
                       n_jobs=5, 
                       scoring=scoring, 
                       fit_params=fit_params,
                       return_train_score=True)
results

2023-09-28 11:41:13.640222: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 11:41:13.642139: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 5. Tune using inter_op_parallelism_threads for best performance.
2023-09-28 11:41:13.657782: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 11:41:13.659254: I tensorflow/core/common_runtime/process_util.cc:

{'fit_time': array([42.55647302, 22.05368304, 23.65045524, 42.48882818, 22.75957942]),
 'score_time': array([0.25814939, 0.52500463, 0.37679505, 0.26170731, 0.44021916]),
 'test_loss_constraint': array([0.00293733, 0.00276875, 0.00239859, 0.00350387, 0.00325889]),
 'train_loss_constraint': array([0.00290694, 0.00291008, 0.00296493, 0.00270413, 0.0031631 ]),
 'test_mse': array([0.00252928, 0.00233012, 0.00189623, 0.00296542, 0.00268035]),
 'train_mse': array([0.00239605, 0.00239411, 0.00241321, 0.00225422, 0.00260918]),
 'test_R2': array([0.92521462, 0.93421866, 0.94492294, 0.93667753, 0.94572059]),
 'train_R2': array([0.94580985, 0.94827208, 0.95900597, 0.93698893, 0.94046265])}

In [3]:
import pandas as pd
df = pd.DataFrame(results)
df.describe()

,fit_time,score_time,test_loss_constraint,train_loss_constraint,test_mse,train_mse,test_R2,train_R2
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,30.701804,0.372375,0.002973,0.002930,0.002480,0.002413,0.937351,0.946108
std,10.805757,0.115340,0.000429,0.000164,0.000400,0.000127,0.008440,0.008458
min,22.053683,0.258149,0.002399,0.002704,0.001896,0.002254,0.925215,0.936989
25%,22.759579,0.261707,0.002769,0.002907,0.002330,0.002394,0.934219,0.940463
50%,23.650455,0.376795,0.002937,0.002910,0.002529,0.002396,0.936678,0.945810
75%,42.488828,0.440219,0.003259,0.002965,0.002680,0.002413,0.944923,0.948272
max,42.556473,0.525005,0.003504,0.003163,0.002965,0.002609,0.945721,0.959006


In [4]:
AMNWt_model = model.build_model()
history = AMNWt_model.fit(model.X_train, model.Y_train, epochs=epoch, batch_size=batch_size, verbose=0)

print("R2 :", model.R2(model.Y_train, AMNWt_model.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model.predict(model.X_test)))

2023-09-28 11:41:56.700244: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 11:41:56.701969: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-09-28 11:41:56.861642: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


R2 : 0.9665659326293337
Q2 : 0.8956346154787257


## Search for hyperparameters

In [5]:
from sklearn.model_selection import RandomizedSearchCV

estimator= KerasRegressor(build_fn=model.build_model, 
                          epochs=epoch, 
                          batch_size=batch_size, 
                          verbose=0)

distributions = dict(batch_size=[7,20],
                     nb_epoch=[2,100],
                    #  hidden_dim=[1,2],
                     )

scoring = {"loss_constraint":make_scorer(model.loss_constraint),
           "mse":make_scorer(model.mse),
           "R2":make_scorer(model.R2),
           }

clf = RandomizedSearchCV(estimator, distributions, random_state=0)
search = clf.fit(model.X_test, model.Y_test)

/home/agiralt/anaconda3/envs/AMN/lib/python3.9/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


## Save and load the model

In [1]:
import tensorflow as tf

# from aMNWtModel import AMNWtModel, RNNCell
# model_class = AMNWtModel
# model_file = "Models/AMNWt_model.keras"



from aMRNNModel import AMRNNModel, RNNCell
model_class = AMRNNModel
model_file = "Models/AMRNN_model.keras"


seed = 10
tf.random.set_seed(seed)
dataset_file = "./Dataset/e_coli_core_UB_100.npz"
objective=['BIOMASS_Ecoli_core_w_GAM']

# Dataset plus model structure
print("---------------------------------------- model ----------------------------------------")
model = model_class(dataset_file=dataset_file, 
                   objective=objective,
                   timestep=4,
                   hidden_dim=50,
                   epochs=50, 
                   verbose=True,
                   batch_size=7)
model.printout()

# Preprocessing
from sklearn.preprocessing import MinMaxScaler,StandardScaler,MaxAbsScaler 
from tools import MaxScaler
scaler= MaxScaler()
model.train_test_split(test_size=0.1, random_state=seed)
model.preprocess(scaler)
model.preprocessing_for_specific_model()

batch_size = 7

# Construct and train an AMNWt model
AMNWt_model = model.build_model()
history = AMNWt_model.fit(model.X_train, model.Y_train, epochs=200, batch_size=batch_size, verbose=0)

print("R2 :", model.R2(model.Y_train, AMNWt_model.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model.predict(model.X_test)))

---------------------------------------- model ----------------------------------------
number of metabolites:  72
filtered measurements size:  1
dataset file: ./Dataset/e_coli_core_UB_100.npz
model type: AMNWt
model medium bound: UB
timestep: 4
training set size (100, 20) (100, 1)
training epochs: 50
training regression: True
training batch size: 7
training validation iter: 0
training early stopping: False


2023-09-28 13:46:23.386122: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 13:46:23.387585: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-09-28 13:46:23.539299: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


R2 : 0.947688494269081
Q2 : 0.862418522911932


In [2]:
from tools import custom_loss

seed = 10
tf.random.set_seed(seed)

# Recreate new model from config file, compile and train it. First test on config.
config = AMNWt_model.get_config()
AMNWt_model_= tf.keras.Model.from_config(config, custom_objects={"RNNCell":RNNCell})
my_mse = custom_loss(model.S, model.P_out,model.P_in)
AMNWt_model_.compile(loss=my_mse,optimizer='adam',metrics=[my_mse])
history = AMNWt_model_.fit(model.X_train, model.Y_train, epochs=200, batch_size=batch_size, verbose=0)
# 
print("R2 :", model.R2(model.Y_train, AMNWt_model_.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model_.predict(model.X_test)))

R2 : 0.947688494269081
Q2 : 0.862418522911932


In [3]:
# Save the train AMNWt_model
tf.keras.models.save_model(AMNWt_model,model_file, overwrite=True, save_format=None, save_traces=True)

In [4]:
from tools import custom_loss

AMNWt_model_ = tf.keras.models.load_model(model_file, custom_objects={"RNNCell":RNNCell,
                                                                               "my_mse":custom_loss(model.S, model.P_out,model.P_in)})


history = AMNWt_model_.fit(model.X_train, model.Y_train, epochs=200, batch_size=batch_size, verbose=0)
print("R2 :", model.R2(model.Y_train, AMNWt_model_.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model_.predict(model.X_test)))

R2 : 0.9858660921974866
Q2 : 0.9417074097957974
